In [23]:
#importing libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

import statsmodels.api as sm
from statsmodels.formula.api import ols

import math

#importing file from csv

mkt = pd.read_csv(r"C:\Users\david\OneDrive\Ambiente de Trabalho\Ficheiro csv para lab 7\marketing_customer_analysis.csv")
mkt


,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2/10/11,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2/12/11,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2/6/11,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2/3/11,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [24]:
# cleaning data 
# processing data

def clean_and_process(df):
    cols = []
    for i in range(len(df.columns)):
        cols.append(df.columns[i].lower().replace(' ','_'))
    mkt.columns = cols    
    #creating dataframes for numerical and categorical variables
    df_numerical = df.select_dtypes(include=[np.number])
    df_categoricals = df.select_dtypes(['object'])
    #normalizing data
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(df_numerical)
    normalized_data = pd.DataFrame(normalized_data, columns=df_numerical.columns)
    dummy_data = pd.get_dummies(df_categoricals.drop(['customer','effective_to_date'], axis=1), drop_first=True)
    #Concating DataFrames
    df_data = pd.concat([df_numerical,dummy_data],axis=1)
    return df_data

In [25]:
# applying the function

mkt_data=clean_and_process(mkt)

In [26]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mkt_data.drop(['total_claim_amount'], axis=1),mkt_data['total_claim_amount'],test_size=0.2)


                                                                                                           

In [27]:
#applying linear regression model
Y = y_train
X = X_train[['income','monthly_premium_auto']]
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     3928.
Date:                Tue, 14 Nov 2023   Prob (F-statistic):               0.00
Time:                        09:01:30   Log-Likelihood:                -49248.
No. Observations:                7307   AIC:                         9.850e+04
Df Residuals:                    7304   BIC:                         9.852e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   61.1597      7.529      8.123      0.000      46.401      75.919
income                  -0.0033   7.87e-05    -42.139      0.000      -0.003      -0.003
monthly_premium_auto     5.3467      0.069     77.338      0.000       5.211       5.482
==============================================================================
Omnibus:                      795.797   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4864.155
Skew:                           0.336   Prob(JB):                         0.00
Kurtosis:                       6.940   Cond. No.                     1.52e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.52e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:

#applying linear regression model
Y = y_train
X = X_train[['income','monthly_premium_auto']]
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()

model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     total_claim_amount   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     3928.
Date:                Tue, 14 Nov 2023   Prob (F-statistic):               0.00
Time:                        09:01:36   Log-Likelihood:                -49248.
No. Observations:                7307   AIC:                         9.850e+04
Df Residuals:                    7304   BIC:                         9.852e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   61.1597      7.529      8.123      0.000      46.401      75.919
income                  -0.0033   7.87e-05    -42.139      0.000      -0.003      -0.003
monthly_premium_auto     5.3467      0.069     77.338      0.000       5.211       5.482
==============================================================================
Omnibus:                      795.797   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4864.155
Skew:                           0.336   Prob(JB):                         0.00
Kurtosis:                       6.940   Cond. No.                     1.52e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.52e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
#model validation

ypred=round(model.predict(X),2)
r2=round(r2_score(Y,ypred),3)
mse=round(mean_squared_error(Y,ypred),3)
rmse=round(math.sqrt(mean_squared_error(Y,ypred)),3)
mae=round(mean_absolute_error(Y,ypred),3)
print("R2:",r2,"MSE:",mse,"RMSE:",rmse,"MAE:",mae)    

R2: 0.518 MSE: 41845.329 RMSE: 204.561 MAE: 146.497


In [30]:
# if we use all the variables in the dataset, the model would improve (R2 from 0,51 to 0,77, MAE from 146 to 95)
# but the number of independent variables would increase from 2 to 50

Y = y_train
X = X_train
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
ypred=round(model.predict(X),2)
r2=round(r2_score(Y,ypred),3)
mse=round(mean_squared_error(Y,ypred),3)
rmse=round(math.sqrt(mean_squared_error(Y,ypred)),3)
mae=round(mean_absolute_error(Y,ypred),3)
print("R2:",r2,"MSE:",mse,"RMSE:",rmse,"MAE:",mae)    

R2: 0.773 MSE: 19727.793 RMSE: 140.456 MAE: 96.051


In [31]:
#let's check which variables should we drop
#identifying which variables are not significant (p-values under 0.05)

pvalues = pd.DataFrame(model.pvalues)
pvalues['variable']=pvalues.index

pvalues.columns = ['pvalue', 'variable']

#filtering only significant variables
pvalues_sig=pvalues[pvalues['pvalue']<0.05]
pvalues_sig

X_sig=X.filter(items=pvalues_sig['variable']).drop(['const'],axis=1)

X_sig.head(3)

,income,monthly_premium_auto,response_Yes,education_High School or Below,employmentstatus_Unemployed,gender_M,location_code_Suburban,location_code_Urban,marital_status_Single,policy_Personal L1,renew_offer_type_Offer2,renew_offer_type_Offer3
8513,58705,69,0,0,0,0,0,0,0,0,0,0
100,0,107,1,0,1,0,1,0,0,0,0,0
5021,12964,65,1,0,0,0,1,0,1,0,0,0


In [32]:
#removing non-significant variables and running the new model
#reduction from 50 to 16 variables
#R2, MSE and MAE remained almost unchanged so the quality of the model remains and the number of variables decreased a lot

X_sig = sm.add_constant(X_sig)
model = sm.OLS(Y,X_sig).fit()

ypred=round(model.predict(X_sig),2)
r2=round(r2_score(Y,ypred),3)
mse=round(mean_squared_error(Y,ypred),3)
rmse=round(math.sqrt(mean_squared_error(Y,ypred)),3)
mae=round(mean_absolute_error(Y,ypred),3)
print("R2:",r2,"MSE:",mse,"RMSE:",rmse,"MAE:",mae)    

R2: 0.77 MSE: 19950.623 RMSE: 141.247 MAE: 96.837


In [36]:

# Extracting numerical columns
mkt_numerical = mkt.select_dtypes(include=[np.number])

# Standardizing numerical columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
stand_data = scaler.fit_transform(mkt_numerical)
stand_data = pd.DataFrame(stand_data, columns=mkt_numerical.columns)
stand_data.head(3)


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,-0.762878,0.612827,-0.703925,1.678099,-1.543287,-0.42225,-0.822648,-0.169640
1,-0.149245,-1.239617,0.022691,-0.208186,-0.217334,-0.42225,2.106160,2.400737
2,0.710636,0.365710,0.429596,0.288205,-0.360680,-0.42225,-0.404247,0.455734


In [39]:

# Extracting numerical columns
mkt_numerical = mkt.select_dtypes(include=[np.number])

# Standardizing numerical columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
stand_data = scaler.fit_transform(mkt_numerical)
stand_data = pd.DataFrame(stand_data, columns=mkt_numerical.columns)

# Now, let's extract the categorical columns and create dummy variables
df_categoricals = mkt.select_dtypes(['object'])
dummy_data = pd.get_dummies(df_categoricals.drop(['customer', 'effective_to_date'], axis=1), drop_first=True)

# Combining standardized numerical data and dummy categorical data
mkt_data2 = pd.concat([stand_data, dummy_data], axis=1)


In [42]:
#standardized model did not improve the result compared to the previous one

# Assuming you've already defined and fitted the model on the training data (X_train, y_train)
# ...

# Fit the model using X_train and y_train
model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

# Now, predict on the test set
X_test = sm.add_constant(X_test)
ypred_test = model.predict(X_test)

# Calculate evaluation metrics for the test set
r2_test = round(r2_score(y_test, ypred_test), 3)
mse_test = round(mean_squared_error(y_test, ypred_test), 3)
rmse_test = round(math.sqrt(mean_squared_error(y_test, ypred_test)), 3)
mae_test = round(mean_absolute_error(y_test, ypred_test), 3)

print("Test Set Performance Metrics:")
print("R2:", r2_test, "MSE:", mse_test, "RMSE:", rmse_test, "MAE:", mae_test)


Test Set Performance Metrics:
R2: 0.768 MSE: 17251.399 RMSE: 131.345 MAE: 93.614
